In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

First, a [new tool](https://pandas-profiling.ydata.ai/docs/master/pages/getting_started/overview.html)

In [2]:
# You'll thank me later

# pip install pandas-profiling==3.2.0

#might through an error abou markupsafe, need to downgrade
# pip install markupsafe==2.0.1

Let's do a quick and dirty eval.

Steps.

1. Data check
    - Type
    - Describe
    - Info


2. Data Quality
    - Nulls
    - Duplicates
    - Incorrect types
  
  
3. TTS

4. StandardScaler

5. OHE

6. Dummy

7. FSM

In [3]:
df = pd.read_csv('./data/imports-85.csv')

In [4]:
df.head()

,make,fuel,aspiration,doors,style,drive,engine,wheelbase,length,width,...,size,fuelsystem,bore,stroke,compratio,horsepower,peakrpm,citympg,himpg,price
0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


In [5]:
# profile = ProfileReport(df, title="cars")

In [6]:
# profile.to_notebook_iframe()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   make        205 non-null    object 
 1   fuel        205 non-null    object 
 2   aspiration  205 non-null    object 
 3   doors       205 non-null    object 
 4   style       205 non-null    object 
 5   drive       205 non-null    object 
 6   engine      205 non-null    object 
 7   wheelbase   205 non-null    float64
 8   length      205 non-null    float64
 9   width       205 non-null    float64
 10  height      205 non-null    float64
 11  weight      205 non-null    int64  
 12  type        205 non-null    object 
 13  cylinders   205 non-null    object 
 14  size        205 non-null    int64  
 15  fuelsystem  205 non-null    object 
 16  bore        205 non-null    object 
 17  stroke      205 non-null    object 
 18  compratio   205 non-null    float64
 19  horsepower  205 non-null    o

In [8]:
df.isnull().sum()

make          0
fuel          0
aspiration    0
doors         0
style         0
drive         0
engine        0
wheelbase     0
length        0
width         0
height        0
weight        0
type          0
cylinders     0
size          0
fuelsystem    0
bore          0
stroke        0
compratio     0
horsepower    0
peakrpm       0
citympg       0
himpg         0
price         0
dtype: int64

In [9]:
df = df.applymap(lambda x: np.nan if x == '?' else x)

In [10]:
df.isnull().sum()

make          0
fuel          0
aspiration    0
doors         2
style         0
drive         0
engine        0
wheelbase     0
length        0
width         0
height        0
weight        0
type          0
cylinders     0
size          0
fuelsystem    0
bore          4
stroke        4
compratio     0
horsepower    2
peakrpm       2
citympg       0
himpg         0
price         4
dtype: int64

In [11]:
df.dropna(inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 0 to 204
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   make        193 non-null    object 
 1   fuel        193 non-null    object 
 2   aspiration  193 non-null    object 
 3   doors       193 non-null    object 
 4   style       193 non-null    object 
 5   drive       193 non-null    object 
 6   engine      193 non-null    object 
 7   wheelbase   193 non-null    float64
 8   length      193 non-null    float64
 9   width       193 non-null    float64
 10  height      193 non-null    float64
 11  weight      193 non-null    int64  
 12  type        193 non-null    object 
 13  cylinders   193 non-null    object 
 14  size        193 non-null    int64  
 15  fuelsystem  193 non-null    object 
 16  bore        193 non-null    object 
 17  stroke      193 non-null    object 
 18  compratio   193 non-null    float64
 19  horsepower  193 non-null    o

In [13]:
cylmap = {
    'four': 4,
    'six': 6,
    'five': 5,
    'eight': 8,
    'three': 3,
    'twelve': 12
}

In [14]:
df.cylinders.replace(cylmap).value_counts()

4     153
6      24
5      10
8       4
12      1
3       1
Name: cylinders, dtype: int64

In [15]:
df.cylinders = df.cylinders.replace(cylmap)

In [16]:
df.horsepower = df.horsepower.astype(int)

In [17]:
df.peakrpm = df.peakrpm.astype(int)

In [18]:
df.doors.value_counts()

four    112
two      81
Name: doors, dtype: int64

In [19]:
df.doors = df.doors.apply(lambda x: 2 if x == 'two' else 4)
df.doors.value_counts()

4    112
2     81
Name: doors, dtype: int64

In [21]:
[col for col in df if df[col].dtypes == 'object']

['make',
 'fuel',
 'aspiration',
 'style',
 'drive',
 'engine',
 'type',
 'fuelsystem',
 'bore',
 'stroke',
 'price']

In [23]:
for col in objectcollist: ohe.fit(X_train.col)

NameError: name 'objectcollist' is not defined